# Setup 


In [6]:
import os
from openai import OpenAI

import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import BaseOutputParser
from langchain.schema.runnable import RunnableLambda
import json
from dotenv import load_dotenv
from langdetect import detect


load_dotenv()

True

In [ ]:
llm = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    temperature=0
)

multilingual_llm = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="Qwen/Qwen3-32B",
    temperature=0
)

# CP 1. Climate legislation Pipeline

##### Indicator CP 1.a Does the country have a framework climate law or equivalent?

Individualy test the following (A country is assessed as ‘Yes’ if it has a framework climate law that fulfils all of
the following):

1. It sets a strategic direction for decarbonisation (i.e. it must include a clear
statement to meet the goals of the Paris Agreement or a national long-term
decarbonisation target)

2. It is enshrined in law (i.e. it must be legislative rather than executive, except in
particular political systems)

3. It sets out at least one of the following obligations: meeting a national target;
developing, revising, implementing or complying with domestic plans,
strategies or policies; developing policy instruments such as regulation, taxation
or public spending in support of climate change goals

4. Also check this In exceptional cases, the combination of a broad environmental law and a clearly
linked executive climate strategy may be sufficient to meet these criteria

**Then have other model check if 1,2,3 are satisfied or 4 is satisfied -> answer yes**

**Then have another model format output into markdown using (this)[https://python.langchain.com/docs/tutorials/extraction/]**



### 1.1 Chain of thought Implementation
#### 1.1.1 Prompting

In [24]:
# Define individual criterion evaluation prompts
cp1a_criterion_1_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to evaluate whether a climate law sets a strategic direction for decarbonisation.
    
    A law meets this criterion if it includes a clear statement to meet the goals of the Paris Agreement 
    OR a national long-term decarbonisation target.
    
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

     
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this law set a strategic direction for decarbonisation?")
])

cp1a_criterion_2_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate whether a climate law is enshrined in law.
    
    A law meets this criterion if it is legislative rather than executive 
    (except in particular political systems where executive action has legal force).
     
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nIs this law enshrined in law (legislative rather than executive)?")
])

cp1a_criterion_3_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate whether a climate law sets out specific obligations.
    
    A law meets this criterion if it sets out at least one of the following:
    - Meeting a national target
    - Developing, revising, implementing or complying with domestic plans, strategies or policies
    - Developing policy instruments such as regulation, taxation or public spending in support of climate goals
    
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this law set out the required obligations?")
])

cp1a_criterion_4_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate the exceptional case criterion.
    
    This criterion is met if there is a combination of:
    - A broad environmental law AND
    - A clearly linked executive climate strategy
    
    This combination may be sufficient to meet the framework criteria in exceptional cases.
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this represent a valid exceptional case (broad environmental law + linked executive strategy)?")
])

# Final assessment prompt
cp1a_final_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst making a final assessment of climate legislation.
    
    A country is assessed as 'YES' for having framework climate law if:
    - Criteria 1, 2, AND 3 are all satisfied, OR
    - Criterion 4 is satisfied (exceptional case)
   
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    Based on the individual assessments, provide a final 'YES' or 'NO' answer with reasoning."""),
    ("human", """Individual criterion assessments:
    Criterion 1 (Strategic direction): {criterion_1_result}
    Criterion 2 (Enshrined in law): {criterion_2_result}
    Criterion 3 (Obligations): {criterion_3_result}
    Criterion 4 (Exceptional case): {criterion_4_result}
    
    What is the final assessment?""")
])

#### 1.1.2 Retrieve Context


In [ ]:
cp1a_1_content = [
  {
    "chunk_content": "The UK’s 2035 Nationally Determined Contribution (NDC) target, announced by the Prime Minister at COP29 in November 2024, is to reduce all greenhouse gas emissions by at least 81% on 1990 levels, excluding international aviation and shipping emissions.",
    "document": "UK’s 2035 Nationally Determined Contribution (NDC) emissions reduction target under the Paris Agreement - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The UK’s 2035 Updated Nationally Determined Contribution confirms the topline ambition of reducing emissions by 81% (based on 1990 levels). This target is in line with advice given by the Climate Change Committee in October 2024, and represents a 1.5°C-aligned, economy-wide absolute emissions reduction target.",
    "document": "The UK's NDC and the Seventh Carbon Budget - Chapter Zero",
    "page_number": 1
  },
  {
    "chunk_content": "The UK government has been advised by the independent Climate Change Committee (CCC) to commit to reducing greenhouse gas emissions by 81% by 2035 compared to 1990 levels. This recommendation aims to be both fair and ambitious in contributing to the Paris Agreement.",
    "document": "UK must raise its climate ambition to restore global reputation, says adviser - Financial Times",
    "page_number": 1
  },
  {
    "chunk_content": "At COP29 in Baku, British PM Keir Starmer announced an ambitious target to reduce the UK's greenhouse gas emissions by at least 81% by 2035, compared to 1990 levels. This is an increase from the previous target of a 68% cut by 2030, with the ultimate goal of achieving carbon neutrality by 2050.",
    "document": "COP29: British PM Starmer wants to regain 'climate leadership' - Le Monde",
    "page_number": 1
  },
  {
    "chunk_content": "The UK submitted its 2035 NDC to the UNFCCC on 30 January 2025. It will also be available on the UNFCCC NDC registry. The UK will produce an updated cross-economy plan in due course, outlining the policies needed to deliver carbon budgets 4-6 and the 2030 and 2035 NDCs on the pathway to net zero by 2050.",
    "document": "UK's 2035 Nationally Determined Contribution (NDC) emissions reduction target under the Paris Agreement - GOV.UK",
    "page_number": 1
  }
]

cp1a_2_content = [
  {
    "chunk_content": "The Climate Change Act 2008 was the first law of its kind. It makes long-term emissions reduction targets legally-binding, opening the door to citizen lawsuits if the government misses them, and introduces interim targets or 'carbon budgets' to keep the UK on track.",
    "document": "Climate Legislation in the United Kingdom - Canadian Climate Institute",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Act, as amended in 2019, commits the UK to ‘net zero’ by 2050. The original act, passed in 2008, committed the UK to an 80% reduction of greenhouse gas emissions by 2050, compared to 1990 levels.",
    "document": "Climate change targets: the road to net zero? - House of Lords Library",
    "page_number": 1
  },
  {
    "chunk_content": "In June 2021, the government set in law the sixth carbon budget (CB6) limiting the volume of greenhouse gases emitted from 2033 to 2037. CB6 reduces emissions by approximately 78% by 2035 compared to 1990 levels.",
    "document": "Why Net Zero - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The UK Parliament passed The Climate Change Act 2008 (2050 Target Amendment) Order 2019, which amends Section 1 of the Climate Change Act 2008 to require the UK to reduce its GHG emissions by 100% compared to their level in 1990.",
    "document": "UK Becomes First G7 Country to Legislate for 'Net-Zero' Emissions - WilmerHale",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Act (2008) made the UK the first country to establish a long-term legally binding framework to cut carbon emissions. It contains a target requiring emissions reductions to Net Zero by 2050.",
    "document": "Climate action - Climate Change Committee",
    "page_number": 1
  }
]

cp1a_3_content = [
  {
    "chunk_content": "The UK’s Net Zero Strategy outlines a comprehensive plan to achieve net zero greenhouse gas emissions by 2050, setting legally binding carbon budgets and detailing sector-specific policies to reduce emissions.",
    "document": "Net Zero Strategy: Build Back Greener - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Act 2008 mandates the UK government to set legally binding carbon budgets, which cap the amount of greenhouse gases emitted over a five-year period, ensuring progress towards the 2050 net zero target.",
    "document": "Climate Change Act 2008 - legislation.gov.uk",
    "page_number": 1
  },
  {
    "chunk_content": "The UK Emissions Trading Scheme (UK ETS), established in 2021, sets a cap on total greenhouse gas emissions and allows trading of emission allowances, providing a market-based approach to incentivize emission reductions.",
    "document": "UK Emissions Trading Scheme - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The Renewables Obligation requires electricity suppliers to source an increasing proportion of electricity from renewable sources, supporting the development of renewable energy technologies through financial incentives.",
    "document": "Renewables Obligation (United Kingdom) - Wikipedia",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Levy is a tax on energy delivered to non-domestic users in the UK, designed to encourage energy efficiency and reduce carbon emissions in the industrial, commercial, and public services sectors.",
    "document": "Climate Change Levy - GOV.UK",
    "page_number": 1
  }
]

cp1a_4_content = [
  {
    "chunk_content": "The Environment Act 2021 provides a broad framework through which specific environmental rules and regulation will operate. While it remains to be seen how the government will implement the Act’s provisions, it represents a significant opportunity for environmental law to drive the UK’s transition to net zero. The Act requires the Government to devise long-term legally binding targets concerning biodiversity, water, air quality, resource efficiency and waste management.",
    "document": "Environmental Law and Climate Change - Centre for Climate Engagement",
    "page_number": 1
  },
  {
    "chunk_content": "The Government’s 25-year environmental plan supports the Act by requiring that all policies, programmes, and investment decisions take due consideration of climate change.",
    "document": "Environmental Law and Climate Change - Centre for Climate Engagement",
    "page_number": 1
  },
  {
    "chunk_content": "The Environment Act 2021, passed in the UK, sets forth new environmental targets and establishes the Office for Environmental Protection (OEP) as an oversight body. It outlines a governance framework and mandates an environmental improvement plan for England and Wales, focusing on enhancing environmental standards and biodiversity.",
    "document": "British environmental law - Wikipedia",
    "page_number": 1
  },
  {
    "chunk_content": "The UK’s Net Zero Strategy outlines a comprehensive plan to achieve net zero greenhouse gas emissions by 2050, setting legally binding carbon budgets and detailing sector-specific policies to reduce emissions.",
    "document": "Net Zero Strategy: Build Back Greener - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The Net Zero Strategy builds on the Government’s 10 point plan for a green industrial revolution which was published on 18 November 2020. It focuses on advancing offshore wind, driving the growth of low carbon hydrogen, delivering new and advanced nuclear power, accelerating the shift to zero emission vehicles, and other key areas.",
    "document": "Government support for a nature and climate declaration - House of Commons Library",
    "page_number": 1
  }
]

cp1a_retrieved_chunks = [cp1a_1_content, cp1a_2_content, cp1a_3_content, cp1a_4_content]


#### 1.1.4 Run

In [ ]:
# Standard chains
cp1a_criterion_1_chain = cp1a_criterion_1_prompt | llm
cp1a_criterion_2_chain = cp1a_criterion_2_prompt | llm
cp1a_criterion_3_chain = cp1a_criterion_3_prompt | llm
cp1a_criterion_4_chain = cp1a_criterion_4_prompt | llm

# Multilingual chains
cp1a_criterion_1_chain_noneng = cp1a_criterion_1_prompt | multilingual_llm
cp1a_criterion_2_chain_noneng = cp1a_criterion_2_prompt | multilingual_llm
cp1a_criterion_3_chain_noneng = cp1a_criterion_3_prompt | multilingual_llm
cp1a_criterion_4_chain_noneng = cp1a_criterion_4_prompt | multilingual_llm


def detect_language(content):
    """Detect language of content (list of chunks or string)"""
    try:
        if isinstance(content, list):
            text = content[0]["chunk_content"] if content else ""
        else:
            text = str(content)
        return detect(text)
    except:
        return 'en'

def evaluate_all_criteria_multilingual(cp1a_retrieved_chunks, return_only_result=False):
    """Evaluate all four criteria using appropriate LLM based on language detection
    
    Args:
        retrieved_chunks: List of 4 elements [cp1a_1_content, cp1a_2_content, cp1a_3_content, cp1a_4_content]
        return_only_result: If True, returns only YES/NO, if False returns full details
    """
    
    # Extract individual content lists
    cp1a_1_content, cp1a_2_content, cp1a_3_content, cp1a_4_content = cp1a_retrieved_chunks
    
    # Detect languages for each context
    lang_1 = detect_language(cp1a_1_content)
    lang_2 = detect_language(cp1a_2_content)
    lang_3 = detect_language(cp1a_3_content)
    lang_4 = detect_language(cp1a_4_content)
    
    # Helper function to choose appropriate chain
    def get_chain_for_language(language, standard_chain, multilingual_chain):
        return multilingual_chain if language != 'en' else standard_chain
    
    # Route to appropriate chains based on language
    chain_1 = get_chain_for_language(lang_1, cp1a_criterion_1_chain, cp1a_criterion_1_chain_noneng)
    chain_2 = get_chain_for_language(lang_2, cp1a_criterion_2_chain, cp1a_criterion_2_chain_noneng)
    chain_3 = get_chain_for_language(lang_3, cp1a_criterion_3_chain, cp1a_criterion_3_chain_noneng)
    chain_4 = get_chain_for_language(lang_4, cp1a_criterion_4_chain, cp1a_criterion_4_chain_noneng)
    
    # Execute evaluations
    cp1a_criterion_1_result = chain_1.invoke({"context": cp1a_1_content})
    cp1a_criterion_2_result = chain_2.invoke({"context": cp1a_2_content})
    cp1a_criterion_3_result = chain_3.invoke({"context": cp1a_3_content})
    cp1a_criterion_4_result = chain_4.invoke({"context": cp1a_4_content})
    
    if return_only_result:
        # Return minimal structure for YES/NO chain
        return {
            "criterion_1_result": cp1a_criterion_1_result.content,
            "criterion_2_result": cp1a_criterion_2_result.content,
            "criterion_3_result": cp1a_criterion_3_result.content,
            "criterion_4_result": cp1a_criterion_4_result.content
        }
    else:
        # Return full details for analysis
        return {
            "criterion_1_result": cp1a_criterion_1_result.content,
            "criterion_2_result": cp1a_criterion_2_result.content,
            "criterion_3_result": cp1a_criterion_3_result.content,
            "criterion_4_result": cp1a_criterion_4_result.content,
            "languages_detected": {
                "context_1": lang_1,
                "context_2": lang_2,
                "context_3": lang_3,
                "context_4": lang_4
            }
        }

def extract_yes_no_result(assessment_result):
    """Extract just YES or NO from the final assessment result"""
    content = assessment_result.content.upper()
    if "YES" in content and "NO" not in content:
        return "YES"
    elif "NO" in content:
        return "NO"
    else:
        return "UNCLEAR"  # Fallback for edge cases

# Two separate chains for different use cases
# Full detailed assessment chain
cp1a_complete_assessment_chain_detailed = (
    RunnableLambda(lambda x: evaluate_all_criteria_multilingual(x, return_only_result=False))
    | cp1a_final_assessment_prompt 
    | llm
)

# Simple YES/NO assessment chain
cp1a_complete_assessment_chain_simple = (
    RunnableLambda(lambda x: evaluate_all_criteria_multilingual(x, return_only_result=True))
    | cp1a_final_assessment_prompt 
    | llm
    | RunnableLambda(extract_yes_no_result)
)

In [26]:
# Usage examples:
# For detailed analysis
detailed_result = cp1a_complete_assessment_chain_detailed.invoke(cp1a_retrieved_chunks)
print("Detailed Assessment:")
print(detailed_result.content)

# For simple YES/NO
simple_result = cp1a_complete_assessment_chain_simple.invoke(cp1a_retrieved_chunks)
print("Simple Assessment:", simple_result)

Detailed Assessment:
Based on the individual criterion assessments, the final assessment is:

**YES**

The UK satisfies the criteria for having a framework climate law because Criterion 1, Criterion 2, AND Criterion 3 are all satisfied. Specifically, the UK has a strategic direction for decarbonisation (page 1, doc UK’s 2035 Nationally Determined Contribution (NDC) emissions reduction target under the Paris Agreement - GOV.UK), the Climate Change Act 2008 is a legislative act (page 1, Climate Legislation in the United Kingdom - Canadian Climate Institute), and the law sets out specific obligations, including meeting a national target of net zero greenhouse gas emissions by 2050 (page 1, Net Zero Strategy: Build Back Greener - GOV.UK).

Additionally, even if the first three criteria were not all satisfied, the UK would still meet the exceptional case criterion (Criterion 4), as it has a broad framework for environmental law (Environment Act 2021, page 1, doc Y) and a comprehensive execu

#### 1.2 Alternative Large context window Implementation

In [ ]:
# Load larger context LLM
large_context_llm = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Llama-3.3-70B-Instruct",
    temperature=0
)

In [22]:
# Set context to whole entire corpus of relevant documetns according to slyvan sheet

all_documents_context = retrieved_chunks

In [ ]:
# Single comprehensive climate legislation assessment prompt
comprehensive_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation assessment. 

Your task is to evaluate whether a country has a framework climate law based on specific criteria and provide a structured markdown assessment.

EVALUATION CRITERIA:
A country is assessed as 'YES' if it has a framework climate law that fulfils ALL of criteria 1, 2, AND 3, OR criterion 4:

1. STRATEGIC DIRECTION: Sets a strategic direction for decarbonisation (must include a clear statement to meet the goals of the Paris Agreement OR a national long-term decarbonisation target)

2. ENSHRINED IN LAW: Is enshrined in law (must be legislative rather than executive, except in particular political systems)

3. OBLIGATIONS: Sets out at least one of the following obligations:
   - Meeting a national target
   - Developing, revising, implementing or complying with domestic plans, strategies or policies
   - Developing policy instruments such as regulation, taxation or public spending in support of climate goals

4. EXCEPTIONAL CASE: The combination of a broad environmental law AND a clearly linked executive climate strategy may be sufficient to meet these criteria

ASSESSMENT LOGIC:
- If criteria 1, 2, AND 3 are all satisfied → YES
- If criterion 4 is satisfied → YES
- Otherwise → NO

OUTPUT FORMAT:
Provide your assessment in the following markdown format:

```markdown
# Climate Legislation Assessment: CP 1.a Framework Climate Law

## Individual Criterion Evaluation

### Criterion 1: Strategic Direction for Decarbonisation
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of whether the law includes clear Paris Agreement goals or long-term decarbonisation targets]

### Criterion 2: Enshrined in Law
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of whether this is legislative rather than executive]

### Criterion 3: Sets Out Obligations
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of which obligations are present, if any]

### Criterion 4: Exceptional Case
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of whether broad environmental law + executive strategy combination exists]

## Final Assessment

**Overall Result:** [YES/NO]

**Logic Applied:** [Explain whether criteria 1+2+3 are satisfied OR criterion 4 is satisfied]

**Conclusion:** [Brief summary of why the country does/does not have a framework climate law]
```"""),
    ("human", "Context: {context}\n\nPlease evaluate whether this country has a framework climate law based on the provided context. For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).")
])

# Create the single-model chain
single_model_assessment_chain = comprehensive_assessment_prompt | large_context_llm


# Run the assessment
result = single_model_assessment_chain.invoke({"context": all_documents_context})
print(result.content)

# Climate Legislation Assessment: CP 1.a Framework Climate Law

## Individual Criterion Evaluation

### Criterion 1: Strategic Direction for Decarbonisation
**Result:** YES
**Reasoning:** The UK has set a strategic direction for decarbonisation through its 2035 Nationally Determined Contribution (NDC) target, aiming to reduce greenhouse gas emissions by at least 81% compared to 1990 levels, which aligns with the goals of the Paris Agreement.

### Criterion 2: Enshrined in Law
**Result:** YES
**Reasoning:** The Climate Change Act 2008, as amended in 2019, commits the UK to 'net zero' by 2050 and makes long-term emissions reduction targets legally binding, indicating that the strategic direction is enshrined in law.

### Criterion 3: Sets Out Obligations
**Result:** YES
**Reasoning:** The Climate Change Act 2008 mandates the UK government to set legally binding carbon budgets, which cap the amount of greenhouse gases emitted over a five-year period, ensuring progress towards the 2050 net

# CP1b - Does the country’s framework climate law specify key accountability elements?


A country is assessed as ‘Yes’ if its framework climate law contains all three of the following accountability elements: 
1. Specification of who is accountable to whom for at least one stated obligation (e.g. accountability of executive to parliament, or private parties to executive authorities) 
2. Specification of how compliance is assessed for at least one stated obligation (e.g. transparency mechanisms in the form of monitoring, reporting and verification, parliamentary oversight, expert assessments, court proceedings) 
3. Specification of what happens in the case of non-compliance for at least one stated obligation (e.g. parliamentary intervention, judicial orders, financial penalties). 

For each of the accountability processes outlined these are examples of the types of relationships that usually qualify for the sub-questions. Each sub-question must be satisfied by at least one of the types of accountability.
1. Who is accountable to whom?
Examples of key relationships include:
a.	executive to parliament 
b.	executive to executive and/or administrative agencies 
c.	national to sub-national and sub-national to national
d.	executive to judiciary
e.	executive to expert bodies
f.	executive to citizens 
g.	private parties to citizens 
h.	private parties to executive authorities.
2. How is compliance assessed?
Some of the most common ways of assessing compliance are:
a.	transparency mechanisms in the form of monitoring, reporting and verification (MRV). Although this type of mechanism does not specify a decision-maker responsible for assessing whether an obligation has been met, this type of process is crucial for ensuring that there is political accountability for the implementation of climate law
b.	parliamentary oversight, which can also be used to assess the effectiveness of the legislation in achieving the stated aims
c.	Expert assessment
d.	Court proceedings
e.	Alternative Dispute Resolution.
3. What happens in the case of non-compliance?
It is relatively rare for countries to specify what happens in the case of non-compliance. Relevant practices identified in the laws can be grouped under certain categories. Some examples are:
a.	Parliamentary intervention
b.	Governmental or ministerial intervention
c.	Judicial orders
d.	Orders and fines by regulators
e.	Court imposed financial penalties
f.	Regulatory financial penalties
g.	Multi-stakeholder agreements.
Note that if a country has multiple laws assessed under the previous indicator, all are considered under this indicator on whether they together contain the three accountability elements above.

### 2.1 Retreive Context

## use sylvan "Climate law, framework law, environmental law" - filter by climate document family column in DB

In [ ]:
cp1b_content_1 = [
  {
    "document": "Climate Change Act 2008",
    "page": "Section 13",
    "excerpt": "The Secretary of State must prepare such proposals and policies as the Secretary of State considers will enable the carbon budgets to be met... and must lay before Parliament a report setting out the proposals and policies for meeting the carbon budgets.",
    "accountability_type": "executive to parliament"
  },
  {
    "document": "Responding to the Climate Change Committee’s (CCC) 2023 Annual Progress Report to Parliament",
    "page": "Section: Recommendation R2022-230",
    "excerpt": "The Secretary of State of the Department for Energy Security and Net Zero is accountable to Parliament for progress against the net zero target.",
    "accountability_type": "executive to parliament"
  },
  {
    "document": "Environment Bill 2020 Policy Statement",
    "page": "Section: Office for Environmental Protection",
    "excerpt": "The Office for Environmental Protection will monitor progress in improving the natural environment in accordance with the government’s domestic environmental improvement plans and targets.",
    "accountability_type": "executive to expert bodies"
  },
  {
    "document": "Net zero and integrated policymaking - House of Lords Library",
    "page": "Section: Devolved administrations",
    "excerpt": "The CCC said that Scotland, Wales and Northern Ireland account for around a fifth of UK emissions and therefore have an 'integral role to play' in reaching the net zero target.",
    "accountability_type": "national to sub-national and sub-national to national"
  },
  {
    "document": "Climate Assembly UK - UK Parliament",
    "page": "Section: Climate Assembly UK",
    "excerpt": "Climate Assembly UK brought together 108 people from across the UK and from all walks of life to examine the question: 'How should the UK meet its target of net zero greenhouse gas emissions by 2050?'",
    "accountability_type": "executive to citizens"
  }
]

cp1b_content_2 = [
  {
    "document": "House of Lords - Methane: keep up the momentum - Environment and Climate Change Committee",
    "page": "Chapter 3: Measurement, monitoring, reporting and verification",
    "excerpt": "The UK has historic and continuing strengths in the monitoring and verification of emissions. Accurate monitoring is essential to track the impact of any mitigation measures. In the UK, methane emissions are estimated annually and published as national statistics in the National Atmospheric Emissions Inventory (NAEI). Each year, the United Kingdom’s National Inventory Report for greenhouse gas emissions is submitted to the United Nations Framework Convention on Climate Change (UNFCCC) in accordance with Intergovernmental Panel on Climate Change (IPCC) guidelines.",
    "compliance_assessment_type": "transparency mechanisms in the form of monitoring, reporting and verification"
  },
  {
    "document": "Environmental audit in the 2019 Parliament - Environmental Audit Committee",
    "page": "Section 2: Sustainability and climate change",
    "excerpt": "We have maintained our scrutiny of Government policies to deliver emissions reductions throughout the Parliament under a workstream entitled Mapping the path to net zero. In the course of this work we have taken evidence regularly from the Climate Change Committee on the findings in its annual reports to Parliament on progress in reducing emissions.",
    "compliance_assessment_type": "parliamentary oversight"
  },
  {
    "document": "Progress in adapting to climate change: 2025 report to Parliament - Climate Change Committee",
    "page": "Box 3.7: Components of the UK Sustainability Disclosure Requirements (SDR)",
    "excerpt": "The Taskforce on Climate-Related Financial Disclosures (TCFD): Mandatory for large UK companies and financial institutions since April 2022, the TCFD framework requires disclosure of climate-related risks, including scenario analysis, under four pillars: governance, strategy, risk management, and metrics and targets.",
    "compliance_assessment_type": "expert assessment"
  },
  {
    "document": "Environment & Climate Change Laws and Regulations Report 2025 United Kingdom",
    "page": "Section 9.3: Emissions trading schemes",
    "excerpt": "In May 2024, ClientEarth, Good Law Project and Friends of the Earth succeeded in their application for judicial review against the Government’s Carbon Budget Delivery Plan (CBDP) on the basis of its conflict with the Climate Change Act 2008.",
    "compliance_assessment_type": "court proceedings"
  },
  {
    "document": "DEB0008 - Evidence on Pre-legislative scrutiny of the Draft Environment (Principles and Governance) Bill",
    "page": "Paragraphs 10-12",
    "excerpt": "The use of fines can secure anticipatory compliance, and if deployed only as a last resort can be a rarely used but powerful weapon in an enforcement body’s armoury to deter offenders. Any income derived from the fine can be hypothecated for particular purposes, for example to support public authorities that are struggling to implement environmental law effectively, or to support other environmental projects and schemes.",
    "compliance_assessment_type": "alternative dispute resolution"
  }
]

cp1b_content_3 = [
  {
    "document": "Environment Agency enforcement and sanctions policy",
    "page": "Section 7.3.2",
    "excerpt": "We may issue variable monetary penalties for more serious offences, including when there is evidence of negligence or mismanagement, when there is an environmental impact, to remove an identifiable financial gain or saving as a result of the breach, or where prosecution is not necessary or can be avoided to get the most appropriate outcome.",
    "non_compliance_consequence": "financial penalties"
  },
  {
    "document": "UK government's climate action plan is unlawful, High Court rules",
    "page": "N/A",
    "excerpt": "The High Court has ruled the UK's latest climate action plan as unlawful. The ruling, which upheld four out of five legal challenges presented by environmental groups, criticized the government's strategy for insufficient risk assessment and over-reliance on unproven technologies like carbon capture and storage. The High Court ordered the government to devise a new plan within 12 months.",
    "non_compliance_consequence": "judicial orders"
  },
  {
    "document": "Drax to pay £25m after reporting failures over wood pellets",
    "page": "N/A",
    "excerpt": "Drax, the owner of Britain’s largest power station, has been fined £25 million by the energy regulator Ofgem for failing to provide accurate sustainability data on wood pellets imported from Canada between April 2021 and March 2022.",
    "non_compliance_consequence": "financial penalties"
  },
  {
    "document": "UK aims to stop water pollution with tougher penalties, stronger regulation",
    "page": "N/A",
    "excerpt": "The UK government is introducing new legislation to tackle water pollution by imposing tougher penalties, including imprisonment for water company executives and granting new powers to the regulator. This response comes after sewage spills reached a record high in 2023, causing public outrage.",
    "non_compliance_consequence": "imprisonment and regulatory penalties"
  },
  {
    "document": "Environment Agency enforcement and sanctions policy",
    "page": "Section 7.1.3",
    "excerpt": "Many of the regimes we enforce contain powers to serve specific enforcement notices. We may serve these where appropriate. These require the recipient to stop offending or to restore or remediate the environment.",
    "non_compliance_consequence": "enforcement notices"
  }
]

cp1b_retrieved_chunks = [
  cp1b_content_1,
  cp1b_content_2,
  cp1b_content_3]

### 2.2 Prompts


In [32]:
# Define individual criterion evaluation prompts
cp1b_criterion_1_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to evaluate whether a country's framework climate **Specification of who is accountable to whom for at least one stated obligation 
    (e.g. accountability of executive to parliament, or private parties to executive authorities)**
    
    These are examples of the types of relationships that usually qualify as having specification of who is accountable to whom:
        a.	executive to parliament 
        b.	executive to executive and/or administrative agencies 
        c.	national to sub-national and sub-national to national
        d.	executive to judiciary
        e.	executive to expert bodies
        f.	executive to citizens 
        g.	private parties to citizens 
        h.	private parties to executive authorities.
    
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this country's framework climate law specify who is accountable to whom for at least one stated obligation?")
])

cp1b_criterion_2_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate whether a country's framework climate law contains the following accountability element:
    **specification of how compliance is assessed for at least one stated obligation 
    (e.g. transparency mechanisms in the form of monitoring, reporting and verification, parliamentary oversight, expert assessments, court proceedings)**
        
    Some of the most common ways of assessing compliance are:
    a.	transparency mechanisms in the form of monitoring, reporting and verification (MRV). Although this type of mechanism does not specify a decision-maker responsible for assessing whether an obligation has been met, this type of process is crucial for ensuring that there is political accountability for the implementation of climate law
    b.	parliamentary oversight, which can also be used to assess the effectiveness of the legislation in achieving the stated aims
    c.	Expert assessment
    d.	Court proceedings
    e.	Alternative Dispute Resolution.
    
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nIs this country's framework climate law specification of how compliance is assessed for at least one stated obligation?")
])

cp1b_criterion_3_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate whether a country's framework climate law contains the following accountability element:
        **Specification of what happens in the case of non-compliance for at least one stated obligation (e.g. parliamentary intervention, judicial orders, financial penalties).**
        
    It is relatively rare for countries to specify what happens in the case of non-compliance. Relevant practices identified in the laws can be grouped under certain categories. Some examples are:
    a.	Parliamentary intervention
    b.	Governmental or ministerial intervention
    c.	Judicial orders
    d.	Orders and fines by regulators
    e.	Court imposed financial penalties
    f.	Regulatory financial penalties
    g.	Multi-stakeholder agreements.
    Note that if a country has multiple laws assessed under the previous indicator, all are considered under this indicator on whether they together contain the three accountability elements above.
    
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this country's framework climate law specify what happens in the case of non-compliance for at least one stated obligation?")
])

# Final assessment prompt
cp1b_final_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst making a final assessment of climate legislation.
    
    A country is assessed as 'YES' if the country's framework climate law specifies ALL THREE of the following key accountability elements:
        Criterion 1: Specification of who is accountable to whom for at least one stated obligation
        Criterion 2: Specification of how compliance is assessed for at least one stated obligation
        Criterion 3: Specification of what happens in the case of non-compliance for at least one stated obligation


    If YES to all criteria, provide a final 'YES' or 'NO' answer with reasoning."""),
    ("human", """Individual criterion assessments:
    Criterion 1 (who is accountable to whom): {criterion_1_result}
    Criterion 2 (compliance is assessed): {criterion_2_result}
    Criterion 3 (case of non-compliance): {criterion_3_result}
    
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    What is the final assessment?""")
])

### 2.3 Implementation

In [ ]:
# Standard chains
cp1b_criterion_1_chain = cp1b_criterion_1_prompt | llm
cp1b_criterion_2_chain = cp1b_criterion_2_prompt | llm
cp1b_criterion_3_chain = cp1b_criterion_3_prompt | llm

# Multilingual chains
cp1b_criterion_1_chain_noneng = cp1b_criterion_1_prompt | multilingual_llm
cp1b_criterion_2_chain_noneng = cp1b_criterion_2_prompt | multilingual_llm
cp1b_criterion_3_chain_noneng = cp1b_criterion_3_prompt | multilingual_llm

def evaluate_all_criteria_multilingual_cp1b(cp1b_retrieved_chunks, return_only_result=False):
    """Evaluate all three accountability criteria using appropriate LLM based on language detection
    
    Args:
        retrieved_chunks: List of 3 elements [cp1b_1_content, cp1b_2_content, cp1b_3_content]
        return_only_result: If True, returns only YES/NO, if False returns full details
    """
    
    # Extract individual content lists
    cp1b_1_content, cp1b_2_content, cp1b_3_content = cp1b_retrieved_chunks
    
    # Detect languages for each context
    lang_1 = detect_language(cp1b_1_content)
    lang_2 = detect_language(cp1b_2_content)
    lang_3 = detect_language(cp1b_3_content)
    
    # Helper function to choose appropriate chain
    def get_chain_for_language(language, standard_chain, multilingual_chain):
        return multilingual_chain if language != 'en' else standard_chain
    
    # Route to appropriate chains based on language
    chain_1 = get_chain_for_language(lang_1, cp1b_criterion_1_chain, cp1b_criterion_1_chain_noneng)
    chain_2 = get_chain_for_language(lang_2, cp1b_criterion_2_chain, cp1b_criterion_2_chain_noneng)
    chain_3 = get_chain_for_language(lang_3, cp1b_criterion_3_chain, cp1b_criterion_3_chain_noneng)
    
    # Execute evaluations
    cp1b_criterion_1_result = chain_1.invoke({"context": cp1b_1_content})
    cp1b_criterion_2_result = chain_2.invoke({"context": cp1b_2_content})
    cp1b_criterion_3_result = chain_3.invoke({"context": cp1b_3_content})
    
    if return_only_result:
        # Return minimal structure for YES/NO chain
        return {
            "criterion_1_result": cp1b_criterion_1_result.content,
            "criterion_2_result": cp1b_criterion_2_result.content,
            "criterion_3_result": cp1b_criterion_3_result.content
        }
    else:
        # Return full details for analysis
        return {
            "criterion_1_result": cp1b_criterion_1_result.content,
            "criterion_2_result": cp1b_criterion_2_result.content,
            "criterion_3_result": cp1b_criterion_3_result.content,
            "languages_detected": {
                "context_1": lang_1,
                "context_2": lang_2,
                "context_3": lang_3
            }
        }

# Two separate chains for different use cases
# Full detailed assessment chain
cp1b_complete_assessment_chain_detailed = (
    RunnableLambda(lambda x: evaluate_all_criteria_multilingual_cp1b(x, return_only_result=False))
    | cp1b_final_assessment_prompt 
    | llm
)

# Simple YES/NO assessment chain
cp1b_complete_assessment_chain_simple = (
    RunnableLambda(lambda x: evaluate_all_criteria_multilingual_cp1b(x, return_only_result=True))
    | cp1b_final_assessment_prompt 
    | llm
    | RunnableLambda(extract_yes_no_result)
)

In [36]:
detailed_result_cp1b = cp1b_complete_assessment_chain_detailed.invoke(cp1b_retrieved_chunks)
print("Detailed Assessment:")
print(detailed_result.content)

# For simple YES/NO
simple_result_cp1b = cp1b_complete_assessment_chain_simple.invoke(cp1b_retrieved_chunks)
print("Simple Assessment:", simple_result)

Detailed Assessment:
Based on the individual criterion assessments, the final assessment is:

**YES**

The UK satisfies the criteria for having a framework climate law because Criterion 1, Criterion 2, AND Criterion 3 are all satisfied. Specifically, the UK has a strategic direction for decarbonisation (page 1, doc UK’s 2035 Nationally Determined Contribution (NDC) emissions reduction target under the Paris Agreement - GOV.UK), the Climate Change Act 2008 is a legislative act (page 1, Climate Legislation in the United Kingdom - Canadian Climate Institute), and the law sets out specific obligations, including meeting a national target of net zero greenhouse gas emissions by 2050 (page 1, Net Zero Strategy: Build Back Greener - GOV.UK).

Additionally, even if the first three criteria were not all satisfied, the UK would still meet the exceptional case criterion (Criterion 4), as it has a broad framework for environmental law (Environment Act 2021, page 1, doc Y) and a comprehensive execu